In [114]:
import pandas as pd
from datetime import datetime, date, timedelta
import weather_scrap as w
import alerts_in_ua as a
from sklearn.preprocessing import LabelEncoder
import holidays

### Merging regions and weather

In [115]:
df_regions = pd.read_csv(f"../data/regions.csv")

In [116]:
df_regions.head(5)

,region,center_city_ua,center_city_en,region_alt,region_id
0,АР Крим,Сімферополь,Simferopol,Крим,1
1,Вінницька,Вінниця,Vinnytsia,Вінниччина,2
2,Волинська,Луцьк,Lutsk,Волинь,3
3,Дніпропетровська,Дніпро,Dnipro,Дніпропетровщина,4
4,Донецька,Донецьк,Donetsk,Донеччина,5


In [117]:
df_regions["region"].unique()

array(['АР Крим', 'Вінницька', 'Волинська', 'Дніпропетровська',
       'Донецька', 'Житомирська', 'Закарпатська', 'Запорізька',
       'Івано-Франківська', 'Київська', 'Кіровоградська', 'Луганська',
       'Львівська', 'Миколаївська', 'Одеська', 'Полтавська', 'Рівненська',
       'Сумська', 'Тернопільська', 'Харківська', 'Херсонська',
       'Хмельницька', 'Черкаська', 'Чернівецька', 'Чернігівська'],
      dtype=object)

In [118]:
df_regions["region"] = df_regions["region"].apply(lambda x: x + " обл.")

In [119]:
weather_regions = [
    "Vinnytsia, Ukraine",
    "Lutsk, Ukraine",
    "Dnipro, Ukraine",
    "Donetsk, Ukraine",
    "Zhytomyr, Ukraine",
    "Uzhhorod, Ukraine",
    "Zaporizhia, Ukraine",
    "Ivano-Frankivsk, Ukraine",
    "Kyiv, Ukraine",
    "Kropyvnytskyi, Ukraine",
    "Lviv, Ukraine",
    "Mykolaiv, Ukraine",
    "Odessa, Ukraine",
    "Poltava, Ukraine",
    "Rivne, Ukraine",
    "Sumy, Ukraine",
    "Ternopil, Ukraine",
    "Kharkiv, Ukraine",
    "Kherson, Ukraine",
    "Khmelnytskyi, Ukraine",
    "Cherkasy, Ukraine",
    "Chernivtsi, Ukraine",
    "Chernigiv, Ukraine"
]

In [120]:
all_data = []

for region in weather_regions:
    forecast = w.generate_forecast(region)
    
    df_region = pd.DataFrame(forecast)
    all_data.append(df_region)

df_weather = pd.concat(all_data, ignore_index=True)
df_weather.head()

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,...,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_uvindex,hour_conditions
0,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,0.0,None,11.2,4.7,146.4,1012.0,24.1,26.3,2.0,Partially cloudy
1,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,0.0,None,11.5,5.4,132.3,1012.0,24.1,33.8,1.0,Partially cloudy
2,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,0.0,None,9.7,4.7,143.2,1013.0,24.1,18.6,0.0,Clear
3,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,0.0,None,9.7,4.3,158.0,1013.0,24.1,23.5,0.0,Partially cloudy
4,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,0.0,None,7.6,3.6,162.7,1013.0,24.1,36.0,0.0,Partially cloudy


In [121]:
df_weather

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,...,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_uvindex,hour_conditions
0,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,0.0,None,11.2,4.7,146.4,1012.0,24.1,26.3,2.0,Partially cloudy
1,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,0.0,None,11.5,5.4,132.3,1012.0,24.1,33.8,1.0,Partially cloudy
2,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,0.0,None,9.7,4.7,143.2,1013.0,24.1,18.6,0.0,Clear
3,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,0.0,None,9.7,4.3,158.0,1013.0,24.1,23.5,0.0,Partially cloudy
4,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,0.0,None,7.6,3.6,162.7,1013.0,24.1,36.0,0.0,Partially cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,"Чернігів, Україна",2025-04-24,1745442000,26.3,8.7,17.5,6.6,52.9,0.0,0.0,...,0.0,None,15.1,5.4,200.0,1009.0,24.1,61.7,8.0,Partially cloudy
548,"Чернігів, Україна",2025-04-24,1745442000,26.3,8.7,17.5,6.6,52.9,0.0,0.0,...,0.0,None,24.1,11.9,297.7,1009.0,24.1,75.4,7.0,Partially cloudy
549,"Чернігів, Україна",2025-04-24,1745442000,26.3,8.7,17.5,6.6,52.9,0.0,0.0,...,0.0,None,25.6,7.9,301.8,1009.0,24.1,93.4,6.0,Overcast
550,"Чернігів, Україна",2025-04-24,1745442000,26.3,8.7,17.5,6.6,52.9,0.0,0.0,...,0.0,None,25.2,11.9,288.3,1008.0,24.1,95.1,5.0,Overcast


In [122]:
df_weather.columns

Index(['city_resolvedAddress', 'day_datetime', 'day_datetimeEpoch',
       'day_tempmax', 'day_tempmin', 'day_temp', 'day_dew', 'day_humidity',
       'day_precip', 'day_precipcover', 'day_solarradiation',
       'day_solarenergy', 'day_uvindex', 'day_sunrise', 'day_sunset',
       'day_moonphase', 'hour_datetime', 'hour_datetimeEpoch', 'hour_temp',
       'hour_humidity', 'hour_dew', 'hour_precip', 'hour_precipprob',
       'hour_snow', 'hour_snowdepth', 'hour_preciptype', 'hour_windgust',
       'hour_windspeed', 'hour_winddir', 'hour_pressure', 'hour_visibility',
       'hour_cloudcover', 'hour_uvindex', 'hour_conditions'],
      dtype='object')

In [123]:
df_weather["city"] = df_weather["city_resolvedAddress"].apply(lambda x: x.split(",")[0])
df_weather["city"] = df_weather["city"].replace('Хмельницька область', "Хмельницький")

In [124]:
df_weather.head(5)

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,...,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_uvindex,hour_conditions,city
0,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,None,11.2,4.7,146.4,1012.0,24.1,26.3,2.0,Partially cloudy,Вінниця
1,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,None,11.5,5.4,132.3,1012.0,24.1,33.8,1.0,Partially cloudy,Вінниця
2,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,None,9.7,4.7,143.2,1013.0,24.1,18.6,0.0,Clear,Вінниця
3,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,None,9.7,4.3,158.0,1013.0,24.1,23.5,0.0,Partially cloudy,Вінниця
4,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,None,7.6,3.6,162.7,1013.0,24.1,36.0,0.0,Partially cloudy,Вінниця


In [125]:
df_weather_reg = pd.merge(df_weather, df_regions, left_on="city",right_on="center_city_ua")

In [126]:
df_weather_reg.head(5)

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,...,hour_visibility,hour_cloudcover,hour_uvindex,hour_conditions,city,region,center_city_ua,center_city_en,region_alt,region_id
0,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,24.1,26.3,2.0,Partially cloudy,Вінниця,Вінницька обл.,Вінниця,Vinnytsia,Вінниччина,2
1,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,24.1,33.8,1.0,Partially cloudy,Вінниця,Вінницька обл.,Вінниця,Vinnytsia,Вінниччина,2
2,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,24.1,18.6,0.0,Clear,Вінниця,Вінницька обл.,Вінниця,Vinnytsia,Вінниччина,2
3,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,24.1,23.5,0.0,Partially cloudy,Вінниця,Вінницька обл.,Вінниця,Vinnytsia,Вінниччина,2
4,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,24.1,36.0,0.0,Partially cloudy,Вінниця,Вінницька обл.,Вінниця,Vinnytsia,Вінниччина,2


In [127]:
df_weather_reg["region"] = df_weather_reg["region"].str.replace(r"\s*обл\.?", " область", regex=True)

In [128]:
kyiv_reg = df_weather_reg[df_weather_reg["region"] == "Київська область"].copy()
kyiv_reg["region"] = "Київ"
kyiv_reg["region_id"] = 1

df_weather_reg = pd.concat([df_weather_reg, kyiv_reg])

### Merging weather, alarms and regions together

In [129]:
df_alarms = pd.DataFrame(a.get_alerts())
df_alarms.head()

,region,status
0,Автономна Республіка Крим,A
1,Волинська область,N
2,Вінницька область,N
3,Дніпропетровська область,N
4,Донецька область,A


In [130]:
df_alarms["status"] = df_alarms["status"].apply(lambda x: 1 if x == 'A' else 0)

In [131]:
df_weather_reg_al = df_weather_reg.merge(df_alarms, how="left", left_on="region", right_on="region")

In [132]:
df_weather_reg_al

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,...,hour_cloudcover,hour_uvindex,hour_conditions,city,region,center_city_ua,center_city_en,region_alt,region_id,status
0,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,26.3,2.0,Partially cloudy,Вінниця,Вінницька область,Вінниця,Vinnytsia,Вінниччина,2,0
1,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,33.8,1.0,Partially cloudy,Вінниця,Вінницька область,Вінниця,Vinnytsia,Вінниччина,2,0
2,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,18.6,0.0,Clear,Вінниця,Вінницька область,Вінниця,Vinnytsia,Вінниччина,2,0
3,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,23.5,0.0,Partially cloudy,Вінниця,Вінницька область,Вінниця,Vinnytsia,Вінниччина,2,0
4,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,36.0,0.0,Partially cloudy,Вінниця,Вінницька область,Вінниця,Vinnytsia,Вінниччина,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,"Київ, Україна",2025-04-24,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,...,58.0,8.0,Partially cloudy,Київ,Київ,Київ,Kyiv,Київщина,1,0
572,"Київ, Україна",2025-04-24,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,...,44.8,5.0,Partially cloudy,Київ,Київ,Київ,Kyiv,Київщина,1,0
573,"Київ, Україна",2025-04-24,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,...,67.3,5.0,Partially cloudy,Київ,Київ,Київ,Kyiv,Київщина,1,0
574,"Київ, Україна",2025-04-24,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,...,71.8,4.0,Partially cloudy,Київ,Київ,Київ,Kyiv,Київщина,1,0


In [133]:
df_weather_reg_al["day_datetime"] = pd.to_datetime(df_weather_reg_al["day_datetime"])

### Merging vectorized ISW to the rest

In [134]:
df_isw_vect = pd.read_csv("../data/ISW_vector.csv")

In [135]:
df_isw_vect.tail(10)

,date,content,lemma_content,stem_content,keywords
1073,2025-02-21,russian offensive campaign assessment 2025 ang...,russian offensive campaign assessment 2025 ang...,russian offens campaign assess 2025 angelica e...,0.521 0.522 0.517 0.845 0.814 0.525 0.583 0.63...
1074,2025-02-22,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.444 0.436 0.422 0.817 0.433 0.621 0.444 0.44...
1075,2025-02-23,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.4 0.396 0.388 0.809 0.409 0.602 0.426 0.4 0....
1076,2025-02-24,russian offensive campaign assessment 2025 ang...,russian offensive campaign assessment 2025 ang...,russian offens campaign assess 2025 angelica e...,0.494 0.493 0.894 0.734 0.493 0.508 0.52 0.482...
1077,2025-02-25,russian offensive campaign assessment 2025 chr...,russian offensive campaign assessment 2025 chr...,russian offens campaign assess 2025 christina ...,0.455 0.464 0.434 0.883 0.463 0.672 0.472 0.45...
1078,2025-02-26,russian offensive campaign assessment 2025 dav...,russian offensive campaign assessment 2025 dav...,russian offens campaign assess 2025 davit gasp...,0.431 0.421 0.425 0.823 0.436 0.661 0.459 0.46...
1079,2025-02-27,russian offensive campaign assessment 2025 chr...,russian offensive campaign assessment 2025 chr...,russian offens campaign assess 2025 christina ...,0.434 0.442 0.433 0.929 0.45 0.643 0.473 0.435...
1080,2025-02-28,russian offensive campaign assessment 2025 oli...,russian offensive campaign assessment 2025 oli...,russian offens campaign assess 2025 olivia gib...,0.518 0.502 0.873 0.758 0.509 0.545 0.511 0.51...
1081,2025-03-01,russian offensive campaign assessment 2025 dav...,russian offensive campaign assessment 2025 dav...,russian offens campaign assess 2025 davit gasp...,0.468 0.466 0.432 0.796 0.69 0.439 0.456 0.492...
1082,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...


In [136]:
#yesterday_str = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')
#df_isw_vect = df_isw_vect[df_isw_vect['date'] == yesterday_str]
df_isw_vect = df_isw_vect.tail(1)

In [137]:
df_isw_vect["date"]

1082    2025-04-21
Name: date, dtype: object

In [138]:
#df_isw_vect["date"] = pd.to_datetime(df_isw_vect["date"])
#df_isw_vect["date_tomorrow_datetime"] = df_isw_vect["date"] + datetime.timedelta(days=1)

In [139]:
#df_ready = df_weather_reg_al.merge(df_isw_vect, how="left", left_on="day_datetime", right_on="date_tomorrow_datetime")
df_isw_vect = pd.concat([df_isw_vect] * len(df_weather_reg_al), ignore_index=True)
df_ready = pd.concat([df_weather_reg_al.reset_index(drop=True), df_isw_vect.reset_index(drop=True)], axis=1)

In [140]:
df_ready.head()

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,...,center_city_ua,center_city_en,region_alt,region_id,status,date,content,lemma_content,stem_content,keywords
0,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,Вінниця,Vinnytsia,Вінниччина,2,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...
1,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,Вінниця,Vinnytsia,Вінниччина,2,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...
2,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,Вінниця,Vinnytsia,Вінниччина,2,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...
3,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,Вінниця,Vinnytsia,Вінниччина,2,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...
4,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,Вінниця,Vinnytsia,Вінниччина,2,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...


In [141]:
df_ready.columns

Index(['city_resolvedAddress', 'day_datetime', 'day_datetimeEpoch',
       'day_tempmax', 'day_tempmin', 'day_temp', 'day_dew', 'day_humidity',
       'day_precip', 'day_precipcover', 'day_solarradiation',
       'day_solarenergy', 'day_uvindex', 'day_sunrise', 'day_sunset',
       'day_moonphase', 'hour_datetime', 'hour_datetimeEpoch', 'hour_temp',
       'hour_humidity', 'hour_dew', 'hour_precip', 'hour_precipprob',
       'hour_snow', 'hour_snowdepth', 'hour_preciptype', 'hour_windgust',
       'hour_windspeed', 'hour_winddir', 'hour_pressure', 'hour_visibility',
       'hour_cloudcover', 'hour_uvindex', 'hour_conditions', 'city', 'region',
       'center_city_ua', 'center_city_en', 'region_alt', 'region_id', 'status',
       'date', 'content', 'lemma_content', 'stem_content', 'keywords'],
      dtype='object')

In [142]:
df_ready

,city_resolvedAddress,day_datetime,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,...,center_city_ua,center_city_en,region_alt,region_id,status,date,content,lemma_content,stem_content,keywords
0,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,Вінниця,Vinnytsia,Вінниччина,2,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...
1,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,Вінниця,Vinnytsia,Вінниччина,2,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...
2,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,Вінниця,Vinnytsia,Вінниччина,2,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...
3,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,Вінниця,Vinnytsia,Вінниччина,2,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...
4,"Вінниця, Україна",2025-04-23,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,...,Вінниця,Vinnytsia,Вінниччина,2,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,"Київ, Україна",2025-04-24,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,...,Київ,Kyiv,Київщина,1,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...
572,"Київ, Україна",2025-04-24,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,...,Київ,Kyiv,Київщина,1,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...
573,"Київ, Україна",2025-04-24,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,...,Київ,Kyiv,Київщина,1,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...
574,"Київ, Україна",2025-04-24,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,...,Київ,Kyiv,Київщина,1,0,2025-04-21,russian offensive campaign assessment 2025 nic...,russian offensive campaign assessment 2025 nic...,russian offens campaign assess 2025 nicol wolk...,0.525 0.527 0.72 0.88 0.533 0.592 0.597 0.554 ...


In [143]:
df = df_ready.fillna(df_ready.median(numeric_only=True))
df['hour_preciptype'] = df['hour_preciptype'].astype(str)

In [144]:
df_encoded = pd.get_dummies(df, columns=['hour_preciptype'], prefix='hour_preciptype')

In [145]:
bool_columns = df_encoded.select_dtypes(include=['bool']).columns
df_encoded[bool_columns] = df_encoded[bool_columns].astype(int)

In [146]:
preciptype = [
    "hour_preciptype_['freezingrain']",
    "hour_preciptype_['ice']",
    "hour_preciptype_['rain', 'snow']",
    "hour_preciptype_['rain']",
    "hour_preciptype_['snow']"
]

for col in preciptype:
    if col not in df_encoded.columns:
        df_encoded[col] = 0
other_cols = [c for c in df_encoded.columns if c not in preciptype]
df_encoded = df_encoded[other_cols + preciptype]

In [147]:
todrop = [
    'city_resolvedAddress',
    'day_datetime', 
    'city', 
    'region',
    'center_city_ua', 
    'center_city_en', 
    'region_alt', 
    'date', 
    'content', 
    'lemma_content', 
    'stem_content']

In [148]:
df_encoded = df_encoded.drop(todrop, axis=1)

In [149]:
df_encoded.columns

Index(['day_datetimeEpoch', 'day_tempmax', 'day_tempmin', 'day_temp',
       'day_dew', 'day_humidity', 'day_precip', 'day_precipcover',
       'day_solarradiation', 'day_solarenergy', 'day_uvindex', 'day_sunrise',
       'day_sunset', 'day_moonphase', 'hour_datetime', 'hour_datetimeEpoch',
       'hour_temp', 'hour_humidity', 'hour_dew', 'hour_precip',
       'hour_precipprob', 'hour_snow', 'hour_snowdepth', 'hour_windgust',
       'hour_windspeed', 'hour_winddir', 'hour_pressure', 'hour_visibility',
       'hour_cloudcover', 'hour_uvindex', 'hour_conditions', 'region_id',
       'status', 'keywords', 'hour_preciptype_None',
       'hour_preciptype_['freezingrain']', 'hour_preciptype_['ice']',
       'hour_preciptype_['rain', 'snow']', 'hour_preciptype_['rain']',
       'hour_preciptype_['snow']'],
      dtype='object')

In [150]:
df_encoded.shape

(576, 40)

In [151]:
temp_df = df_encoded['keywords'].str.split(expand=True)
temp_df = temp_df.astype('float64')
df_encoded_v2 = pd.concat([df_encoded.drop('keywords', axis=1), temp_df], axis=1)

In [152]:
unique_values = sorted(df['hour_conditions'].astype(str).unique())
mapping = {val: idx for idx, val in enumerate(unique_values)}
df_encoded_v2['hour_conditions'] = df['hour_conditions'].astype(str).map(mapping)

In [153]:
df_encoded_v2['hour_datetime'] = pd.to_datetime(df_encoded_v2['hour_datetime'], format='%H:%M:%S')
df_encoded_v2['day_sunrise'] = pd.to_datetime(df_encoded_v2['day_sunrise'], format='%H:%M:%S')
df_encoded_v2['day_sunset'] = pd.to_datetime(df_encoded_v2['day_sunset'], format='%H:%M:%S')
df_encoded_v2['hour_datetime'] = (df_encoded_v2['hour_datetime'].dt.hour * 3600 +
                                  df_encoded_v2['hour_datetime'].dt.minute * 60 +
                                  df_encoded_v2['hour_datetime'].dt.second) / 86400

df_encoded_v2['day_sunrise'] = (df_encoded_v2['day_sunrise'].dt.hour * 3600 +
                                df_encoded_v2['day_sunrise'].dt.minute * 60 +
                                df_encoded_v2['day_sunrise'].dt.second) / 86400

df_encoded_v2['day_sunset'] = (df_encoded_v2['day_sunset'].dt.hour * 3600 +
                               df_encoded_v2['day_sunset'].dt.minute * 60 +
                               df_encoded_v2['day_sunset'].dt.second) / 86400

In [154]:
df_encoded_v2

,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,...,990,991,992,993,994,995,996,997,998,999
0,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,24.8,...,0.659,0.853,0.708,0.697,0.585,0.665,0.689,0.64,0.658,0.803
1,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,24.8,...,0.659,0.853,0.708,0.697,0.585,0.665,0.689,0.64,0.658,0.803
2,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,24.8,...,0.659,0.853,0.708,0.697,0.585,0.665,0.689,0.64,0.658,0.803
3,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,24.8,...,0.659,0.853,0.708,0.697,0.585,0.665,0.689,0.64,0.658,0.803
4,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,24.8,...,0.659,0.853,0.708,0.697,0.585,0.665,0.689,0.64,0.658,0.803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,229.1,19.7,...,0.659,0.853,0.708,0.697,0.585,0.665,0.689,0.64,0.658,0.803
572,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,229.1,19.7,...,0.659,0.853,0.708,0.697,0.585,0.665,0.689,0.64,0.658,0.803
573,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,229.1,19.7,...,0.659,0.853,0.708,0.697,0.585,0.665,0.689,0.64,0.658,0.803
574,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,229.1,19.7,...,0.659,0.853,0.708,0.697,0.585,0.665,0.689,0.64,0.658,0.803


In [155]:
df_encoded_v2['timestamp'] = pd.to_datetime(df_encoded_v2['hour_datetimeEpoch'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
df_encoded_v2['timestamp'] = pd.to_datetime(df_encoded_v2['timestamp'])

In [156]:
df_encoded_v2 = df_encoded_v2.set_index(pd.DatetimeIndex(df_encoded_v2['timestamp']))

In [157]:
hourly_has_event = df_encoded_v2.groupby('region_id')['status'].resample('h').sum().gt(0).astype(int)
hours_with_events_per_day = hourly_has_event.groupby('region_id').resample('D', level=1).sum()

hours_with_events_per_day = hours_with_events_per_day.reset_index()

hours_with_events_per_day.rename(columns={'status': 'event_lastDay_region'}, inplace=True)

df_encoded_v2['date'] = df_encoded_v2['timestamp'].dt.date
hours_with_events_per_day['date'] = hours_with_events_per_day['timestamp'].dt.date

df_encoded_v2 = df_encoded_v2.merge(
    hours_with_events_per_day[['region_id', 'date', 'event_lastDay_region']],
    on=['region_id', 'date'],
    how='left'
)

In [158]:
class UkrainianECBHolidays(holidays.HolidayBase):
    def __init__(self, years=None, **kwargs):
        self.country = "UA"
        super().__init__(years=years, **kwargs)

    def _populate(self, year):
        self[date(year, 1, 1)] = "New Year's Day"
        self[date(year, 1, 7)] = "Orthodox Christmas"
        self[date(year, 3, 8)] = "International Women's Day"
        self[date(year, 5, 1)] = "Labour Day"
        self[date(year, 5, 8)] = "Day of Remembrance and Reconciliation"
        self[date(year, 5, 9)] = "Victory Day"
        self[date(year, 6, 28)] = "Constitution Day of Ukraine"
        self[date(year, 8, 24)] = "Independence Day of Ukraine"
        self[date(year, 12, 25)] = "Christmas (Western)"


class RussianECBHolidays(holidays.HolidayBase):
    def __init__(self, years=None, **kwargs):
        self.country = "RU"
        super().__init__(years=years, **kwargs)

    def _populate(self, year):
        self[date(year, 1, 1)] = "New Year's Day (Russia)"
        self[date(year, 1, 7)] = "Orthodox Christmas (Russia)"
        self[date(year, 2, 23)] = "Defender of the Fatherland Day (Russia)"
        self[date(year, 3, 8)] = "International Women's Day (Russia)"
        self[date(year, 5, 1)] = "Spring and Labor Day (Russia)"
        self[date(year, 5, 9)] = "Victory Day (Russia)"
        self[date(year, 6, 12)] = "Russia Day"
        self[date(year, 11, 4)] = "Unity Day (Russia)"

In [159]:
ua_holidays = UkrainianECBHolidays(years=range(2020, 2032))
ru_holidays = RussianECBHolidays(years=range(2020, 2032))

In [160]:
df_encoded_v2['ru_holiday'] = df_encoded_v2['date'].apply(lambda x: 1 if x in ru_holidays else 0)
df_encoded_v2['ua_holiday'] = df_encoded_v2['date'].apply(lambda x: 1 if x in ua_holidays else 0)

In [161]:
df_encoded_v2.drop('date', axis=1, inplace=True)

In [162]:
df_encoded_v2.drop('timestamp', axis=1, inplace=True)

In [163]:
df_encoded_v2

,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,...,993,994,995,996,997,998,999,event_lastDay_region,ru_holiday,ua_holiday
0,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,24.8,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
1,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,24.8,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
2,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,24.8,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
3,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,24.8,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
4,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,24.8,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,229.1,19.7,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
572,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,229.1,19.7,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
573,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,229.1,19.7,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
574,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,229.1,19.7,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0


In [164]:
col_data = df_encoded_v2.pop("status")
insert_at = df_encoded_v2.columns.get_loc("event_lastDay_region") + 1
df_encoded_v2.insert(insert_at, "status", col_data)

In [165]:
col_data = df_encoded_v2.pop("region_id")
insert_at = df_encoded_v2.columns.get_loc('day_datetimeEpoch')
df_encoded_v2.insert(insert_at, "region_id", col_data)

In [166]:
df_encoded_v2.drop(["hour_preciptype_None"], axis=1, inplace=True)

In [167]:
df_encoded_v2.drop("event_lastDay_region", axis=1, inplace=True)

In [168]:
df_encoded_v2.to_csv("../data/everyhour_predict/test.csv", sep=";", index=False)

In [169]:
df_encoded_v2

,region_id,day_datetimeEpoch,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,...,993,994,995,996,997,998,999,status,ru_holiday,ua_holiday
0,2,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
1,2,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
2,2,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
3,2,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
4,2,1745355600,25.3,12.8,18.7,4.6,42.2,0.0,0.0,286.2,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,1,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,229.1,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
572,1,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,229.1,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
573,1,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,229.1,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
574,1,1745442000,26.8,14.2,20.5,3.6,35.5,0.0,0.0,229.1,...,0.697,0.585,0.665,0.689,0.64,0.658,0.803,0,0,0
